# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [6]:
!pip install langchain langchain-openai langchain-community pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.51
    Uninstalling langchain-core-0.3.51:
      Successfully uninstalled langchain-core-0.3.51


In [25]:
!pip install -r "/content/requirements.txt"
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.4/644.4 kB 42.4 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: openai
    Found existing installation: openai 1.72.0
    Uninstalling openai-1.72.0:
      Successfully uninstalled openai-1.72.0
  Attempting uninstall: langchain-openai
    Found existing installation: langchain-openai 0.3.13
    Uninstalling langchain-openai-0.3.13:
      Successfully uninstalled langchain-openai-0.3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googl

In [35]:
import langchain_openai as loai
from openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain import FewShotPromptTemplate
from langchain.callbacks import get_openai_callback
import re
import os

In [36]:
import warnings
warnings.filterwarnings('ignore')

In [37]:
load_dotenv('/.env')

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

print("OPENAI Key loaded:", OPENAI_API_KEY is not None)
print("HuggingFace Key loaded:", HUGGINGFACEHUB_API_TOKEN is not None)

OPENAI Key loaded: True
HuggingFace Key loaded: True


In [38]:
import pandas as pd
df = pd.read_csv('/content/Data.csv')

In [39]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [79]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [80]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.7)

In [136]:
prompt = ChatPromptTemplate.from_template( #Write a query that would take a variable to describe any product
    "Write a detailed product description for the following product: {product}"
)

In [137]:
chain = LLMChain(llm=llm, prompt=prompt)

In [138]:
response = chain.invoke({"product": "a pair of stylish sneakers"})
print(response["text"])

Introducing our latest pair of stylish sneakers, the perfect blend of fashion and comfort for any occasion. These sneakers are crafted with high-quality materials, ensuring durability and long-lasting wear. The sleek design features a trendy silhouette with a lace-up closure, making them easy to slip on and off.

The sneakers are designed with a cushioned insole for maximum comfort throughout the day, whether you're running errands or out for a night on the town. The rubber sole provides excellent grip and traction, making them perfect for any terrain.

Available in a variety of colors and sizes, these sneakers are sure to complement any outfit. Pair them with jeans for a casual look or dress them up with a skirt for a more refined style. The versatile design makes them a must-have in any shoe collection.

Don't sacrifice style for comfort - with our stylish sneakers, you can have both. Upgrade your footwear game with these chic and practical sneakers that are sure to become a staple i

## SimpleSequentialChain

In [84]:
from langchain.chains import SimpleSequentialChain

In [139]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Generate one creative and unique marketing campaign idea for the following product. Be concise and focus on a single strong concept:\n\nProduct: {product}"

)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [140]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write an engaging and persuasive marketing advertisement for this product, using the review summary to highlight its main benefits and unique selling features: {input}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [141]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [142]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"Timeless Elegance: A Watch Fit for Royalty"

Campaign Idea: Partner with a high-end jewelry store to host an exclusive event where customers can experience the luxury watch collection in a setting fit for royalty. Invite influencers and celebrities to showcase the watches on social media, creating buzz and desire among luxury consumers. Offer limited edition pieces and special discounts to drive sales during the event. Create a sense of exclusivity and luxury that aligns with the brand's image, positioning the watch as a must-have accessory for those who appreciate quality and sophistication.
Introducing "Timeless Elegance: A Watch Fit for Royalty" - the ultimate statement piece for those who seek luxury and sophistication. This exquisite collection of watches is crafted with precision and attention to detail, making each timepiece a work of art that exudes elegance and class.

The reviews are in, and customers are raving about the super

'Introducing "Timeless Elegance: A Watch Fit for Royalty" - the ultimate statement piece for those who seek luxury and sophistication. This exquisite collection of watches is crafted with precision and attention to detail, making each timepiece a work of art that exudes elegance and class.\n\nThe reviews are in, and customers are raving about the superior quality and timeless design of our watches. With a focus on luxury materials and expert craftsmanship, these watches are not just accessories - they are heirlooms to be cherished for generations to come.\n\nImagine yourself wearing a watch fit for royalty, a symbol of refined taste and impeccable style. With our exclusive event at a high-end jewelry store, you have the opportunity to experience these stunning timepieces in person, surrounded by luxury and opulence.\n\nJoin us at our event, where influencers and celebrities will showcase the watches on social media, creating a buzz that will leave luxury consumers craving to own one of

**Repeat the above twice for different products**

In [143]:
# List of products and their reviews
products_to_process = [
    {"product": "Noise-cancelling headphones", "review": "These headphones have great sound quality but the noise cancellation could be better."},
    {"product": "Smart water bottle", "review": "This bottle keeps my drinks cold all day, but it's a little heavy."},
    {"product": "High-quality coffee machine", "review": "This coffee machine brews delicious coffee quickly but leaks sometimes and doesn't keep the coffee hot enough."},
    {"product": "Luxury watch", "review": "This watch has a beautiful design, but the battery life is shorter than expected."}
]

# Iterate over each product and review, applying the SimpleSequentialChain
for item in products_to_process:
    product = item["product"]
    review = item["review"]

    # Process the product using the SimpleSequentialChain
    print(f"Exploring {product}...\n")

    # Generate review summary and marketing ad
    result = overall_simple_chain.run(review)

    # Print the result for each product
    print(f"Marketing Ad for {product}:")
    print(result)
    print("\n====================\n")

Exploring Noise-cancelling headphones...



> Entering new SimpleSequentialChain chain...
Campaign idea: "Tune out the world, turn up the music"

Concept: Highlight the exceptional sound quality of the headphones while acknowledging the room for improvement in noise cancellation. Position the headphones as the ultimate escape from the outside world, encouraging consumers to immerse themselves in their music and block out distractions. Partner with popular music artists and influencers to create content showcasing the transformative power of the headphones. Offer a limited-time discount for those looking to block out the noise and focus on what truly matters: the music.
Are you tired of the noisy world around you constantly distracting you from enjoying your music? Look no further than our revolutionary headphones that will transport you to a world of pure sound bliss. 

With exceptional sound quality that will make you feel like you're front row at a live concert, our headphones will e

## SequentialChain

In [157]:
from langchain.chains import SequentialChain

In [180]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
        "Translate to English only if the review isn't in English. If it is, simply retain the original text:\n\n{review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="english_review"
                     )


In [181]:
second_prompt = ChatPromptTemplate.from_template(
        "Summarize the following product review in 2-3 concise sentences:\n\n{english_review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary"
                    )


In [182]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Detect the original language of the following review:\n\n{review}"

)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="detected_language"
                      )


In [188]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
          "Write a friendly follow-up message to the user who wrote this review. Thank them for their feedback and mention one key point from the summary:\n\nSummary: {summary}\n Original Language: {detected_language}"

)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [189]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["english_review", "summary", "detected_language", "followup_message"],
    verbose=True
)

In [190]:
result = overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


In [191]:
print(f"Original Review: {result['review']}")
print(f"English Translation: {result['english_review']}")
print(f"Summary: {result['summary']}")
print(f"Detected Language: {result['detected_language']}")
print(f"Follow-up Message: {result['followup_message']}")

Original Review: Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...
Vieux lot ou contrefaçon !?
English Translation: I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit !?
Summary: The reviewer finds the taste of the product to be mediocre and dislikes that the foam doesn't hold well. They suspect that the product may be either an old batch or counterfeit compared to ones purchased in stores with better taste.
Detected Language: French
Follow-up Message: Bonjour! 

Thank you so much for taking the time to leave us a review. We truly appreciate your feedback. We're sorry to hear that you found the taste of the product to be mediocre and that the foam doesn't hold well. 

We take the quality of our products very seriously and we assure you that we strive to deliver the best experience to our customers. We will

In [ ]:
# === Example Product Reviews (you can change them) ===
reviews = [
    "La batería dura muchísimo. Me encantó el diseño y los materiales se sienten premium.",
    "I absolutely love this blender! It crushes ice like a beast and makes perfect smoothies.",
    "Pas mal pour le prix, mais la qualité du son pourrait être meilleure.",
    "هذه الساعة الذكية رائعة وتعمل بشكل ممتاز مع هاتفي الأندرويد."
]

# === Run Chain for Each Review ===
for i, review in enumerate(reviews, start=1):
    print(f"\n{'='*20} Product Review {i} {'='*20}")
    result = overall_chain({"review": review})
    print(f"📝 Original Review: {review}")
    print(f"🌐 English Translation: {result['english_review']}")
    print(f"📝 Summary: {result['summary']}")
    print(f"🗣️ Detected Language: {result['detected_language']}")
    print(f"📩 Follow-up Message: {result['followup_message']}")

**Repeat the above twice for different products or reviews**

In [199]:
# Define two sample reviews (one English, one in another language)
reviews = [
    "Este producto superó mis expectativas. La calidad es excelente y el envío fue muy rápido.",
    "The headphones are decent for the price, but the battery life could be better.",
    "هذه الساعة الذكية رائعة وتعمل بشكل ممتاز مع هاتفي الأندرويد."

]

# Run the chain on both reviews
for i, review in enumerate(reviews, start=1):
    print(f"\n{'='*30} Review {i} {'='*30}")
    result = overall_chain({"review": review})
    print(f"Original Review: {review}")
    print(f"English Translation: {result['english_review']}")
    print(f"Summary: {result['summary']}")
    print(f"Detected Language: {result['detected_language']}")
    print(f"Follow-up Message: {result['followup_message']}")



============================== Review 1 ==============================


> Entering new SequentialChain chain...

> Finished chain.
Original Review: Este producto superó mis expectativas. La calidad es excelente y el envío fue muy rápido.
English Translation: This product exceeded my expectations. The quality is excellent and the shipping was very fast.
Summary: The reviewer was pleasantly surprised by the product's quality and fast shipping, finding it to exceed their expectations overall.
Detected Language: Spanish
Follow-up Message: Hola [User],

¡Gracias por tomarte el tiempo para dejarnos tu opinión! Nos alegra mucho saber que quedaste sorprendido por la calidad del producto y la rapidez del envío. En [Company], nos esforzamos por superar las expectativas de nuestros clientes en cada pedido. 

Esperamos que disfrutes mucho tu compra y que sigas confiando en nosotros para futuras adquisiciones.

¡Que tengas un excelente día!

Saludos cordiales,
[Your Name]
[Company]

=============


## Router Chain

In [200]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [201]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [202]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [203]:
llm = ChatOpenAI(temperature=0)

In [204]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [205]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt
)

In [206]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [207]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [208]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [209]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body is characterized by a continuous spectrum of wavelengths and intensities, which depends only on the temperature of the body. This phenomenon is described by Planck's law of black body radiation, which states that the intensity of radiation emitted by a black body at a given wavelength is proportional to the temperature of the body and follows a specific distribution known as the Planck distribution."

In [210]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'2 + 2 is equal to 4.'

In [211]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells, maintaining the integrity and continuity of the organism's genetic code.\n\nAdditionally, DNA serves as a storage system for genetic information that can be accessed and utilized by cells as needed. This allows for the regulation of gene expression, the repair of damaged DNA, and the adaptation to changing environmental conditions.\n\nIn summary, every cell in our body contains DNA because it is essential for the pro

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [219]:
chain.run("What would happen if we could time travel but only to the future? How would this impact our world?")



> Entering new MultiPromptChain chain...
None: {'input': 'What would happen if we could time travel but only to the future? How would this impact our world?'}
> Finished chain.


'If we could only time travel to the future, it would have a significant impact on our world in several ways:\n\n1. Scientific advancements: People could travel to the future to see how technology has evolved and bring back knowledge of future inventions and discoveries. This could accelerate scientific progress and lead to rapid advancements in various fields.\n\n2. Economic implications: Investors and businesses could use time travel to predict future market trends and make strategic decisions based on that information. This could potentially lead to economic instability and inequality as those with access to time travel technology gain an unfair advantage.\n\n3. Social consequences: Time travel to the future could disrupt the natural progression of society and culture. People may become obsessed with the future and neglect the present, leading to a loss of appreciation for the present moment and the value of living in the now.\n\n4. Ethical dilemmas: Time travel to the future could 

In [220]:
chain.run('What if all humans had a superpower? How would society function differently?')



> Entering new MultiPromptChain chain...
None: {'input': 'What if all humans had a superpower? How would society function differently?'}
> Finished chain.


'If all humans had a superpower, society would likely function very differently. People would have a wide range of abilities, from super strength to telekinesis to the ability to fly. This would likely lead to a more equal society, as everyone would have some sort of advantage or ability that could be used to help others.\n\nThere would likely be a shift in power dynamics, as those with more powerful abilities may be seen as leaders or influencers in society. This could potentially lead to conflicts and power struggles as people vie for control or dominance.\n\nOn the other hand, having superpowers could also lead to a greater sense of unity and cooperation among people, as they work together to harness their abilities for the greater good. Society may become more advanced and innovative, as people use their powers to solve problems and improve the world around them.\n\nOverall, a society where all humans have superpowers would be a complex and dynamic one, with both positive and negat

In [221]:
chain.run("What if the Earth suddenly stopped rotating for a second? What would happen to the planet?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What would happen to the Earth if it suddenly stopped rotating for a second?'}
> Finished chain.


"If the Earth were to suddenly stop rotating for a second, everything on the surface of the Earth would continue moving at the same speed as the Earth's rotation. This would result in catastrophic winds and storms as the atmosphere and oceans try to adjust to the sudden change in rotation. Buildings, trees, and anything not securely anchored to the ground would be swept away by the powerful winds. The effects would be devastating, but fortunately, the Earth's rotation is very stable and unlikely to suddenly stop."